# Dependences, tests

In [ ]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy
!pip install einops # Falcon dependency
!pip install huggingface_hub["cli"] # To empty model cache

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-jfht7cax
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jfht7cax
  Resolved https://github.com/huggingface/transformers.git to commit f92cc7034a49959b247a46a210b912e56a6f977d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-tdflwk46
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-tdflwk46


In [12]:
!nvidia-smi
!free

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Aug 23 13:03:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:00:07.0 Off |                    0 |
| N/

In [ ]:
# Empty model cache
!huggingface-cli delete-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
? Select revisions to delete: 0 revisions selected counting for 0.0. 
❯ ○ None of the following (if selected, nothing will be deleted).
 
Model bigscience/mt0-xxl (55.8G, used 6 days ago)
  ○ b5461b49: main # modified 6 days ago
 
Model tiiuae/falcon-40b-instruct (50.3K, used 6 days ago)
  ○ ca78eac0: main # modified 6 days ago
 
Model ichitaka/falcon-40b-instruct-8bit (41.9G, used 6 days ago)
  ○ 03dd12af: main # modified 6 days ago
 
Model Salesforce/xgen-7b-8k-inst (27.6G, used 2 days ago)
  ○ 68f77dec: (detached) # modified 6 days ago
  ○ d008fe87: main # modified 2 days ago
 
Model tiiuae/falcon-7b-instruct (14.4G, used 5 hours ago)
  ○ eb410fb6: main # modified 5 hours ago
 
Model mosaicml/mpt-7b-instruct (13.3G, used 19 minutes ago)
  ○ 925e0d80: main # modified 19 minutes ago
i49mPress <s

# Imports

In [1]:
import time, os, traceback, datetime, torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

### Generation Settings

In [2]:
# Does not work.
#model.generation_config.do_sample = False
#model.generation_config.max_new_tokens = 2048
#model.generation_config.temperature = .3
#model.generation_config.top_k = 100

DO_SAMPLE = True
MAX_NEW_TOKENS = 2048
TEMPERATURE = .3
TOP_K = 100

# Loading the model

### >>> XGen 7B Instruct

In [3]:
model_name = 'Salesforce/xgen-7b-8k-inst'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [2]:
PROMPT_TEMPLATE_1 = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Summarize the following text.

P1: You're finally here! What took so long?
P2: I got stuck in traffic again. There was a terrible traffic jam near the Carrefour intersection.
P1: It's always rather congested down there during rush hour. Maybe you should try to find a different route to get home.
P2: I don't think it can be avoided, to be honest.
P1: perhaps it would be better if you started taking public transport system to work.
P2: I think it's something that I'll have to consider. The public transport system is pretty good.
P1: It would be better for the environment, too.
P2: I know. I feel bad about how much my car is adding to the pollution problem in this city.
P1: Taking the subway would be a lot less stressful than driving as well.
P2: The only problem is that I'm going to really miss having the freedom that you have with a car.
P1: Well, when it's nicer outside, you can start biking to work. That will give you just as much freedom as your car usually provides.
P2: That's true. I could certainly use the exercise!
P1: So, are you going to quit driving to work then?
P2: Yes, it's not good for me or for the environment.

### Assistant: Two individuals discuss the delay caused by traffic congestion near Carrefour intersection. One suggests finding an alternate route or using public transport. They discuss the benefits of public transport for the environment, stress reduction, and consider biking as an option in good weather. Eventually, one person decides to stop driving to work due to health and environmental concerns.

### Human: Summarize the following text.

AC: Once search-and-rescue efforts are under control in China, building experts are going to begin looking at the extent of structural damage. Stanford University's Ann Kiremidjian is a structural engineer. She helped assess damage years after China's 1976 huge earthquake that killed about 250,000 people. Ann Kiremidjian, when you look at the pictures and video of the collapsed buildings around Chengdu, I bet you see things that I don't notice.
Pr. AK: Alex, what I have seen from the video clips is first of all a lot of concrete buildings, what we call concrete frames, and I would classify them as a non-ductile concrete frame. What that means is that there is very little steel reinforcement in the concrete and as a result, the concrete cannot resist displacement in the horizontal direction.
AC: It needs to have steel bars, I think that is called rebar. Yeah.
Pr. AK: That is correct. Not enough rebar both in the vertical direction and in the horizontal direction. And in addition, you have to provide rebar going from the column to the beam at the joint to provide continuity and a way to transfer the forces from one member to the other.
AC: You inspected buildings after this last, huge, terrible earthquake more than 30 years ago. Did you make recommendations to the Chinese government then?
Pr. AK: At the time, they didn't have earthquake-resistant codes in place. And since then, we've had numerous discussions, and the Chinese have had very strong programs to develop seismic codes, to develop seismic hazard maps. The seismic hazard maps similar to the ones that we have in the United States, are developed with the purpose to identify regions of frequent large earthquakes that can cause damage to buildings, the very first step in any earthquake-resistant design.
AC: And what about this region around Chengdu, the Sichuan province, is this prone to earthquakes?
Pr. AK: Yes, all of China is in a highly seismic area. You would notice that there is a fault that runs along the plateau between the Himalayas and where Chengdu is. If you trace all the aftershocks, you can probably trace the fault line that has ruptured. To cause a magnitude 7.8 earthquake, you will need to rupture more than 100 kilometers of a fault, and it looks it is almost 200 kilometers of locations of aftershock which is likely to be the length of the rupture of the fault.
AC: Is there some way to measure the size of the earthquake and then overlay that on a map of the region and figure out, we have to look at every single building within a particular radius from the epicenter? I mean do you have to do that?
Pr. AK: Oh, we do that all the time. In fact, what typically you would do is you would focus within the first 15 or 20 kilometers from the rupture zone. It's not just the epicenter, but it is the rupture zone, and the rupture can run about a couple of hundred kilometers, as it is in the case of China. So you are talking a lot of buildings. A single inspector or 10 or 20 inspectors cannot perform that task in a reasonable amount of time. What you have to do is recruit all the professional structural engineers to help with the inspection process.
AC: Ann, you're speaking to us from your home in Los Altos Hills, that's south of San Francisco. And it's two miles from the most notorious earthquake fault in North America, the San Andreas Fault.
Pr. AK: Correct.
AC: Can you explain to me why a structural engineer would build a house there?
Pr. AK: Yes, indeed. You have to take certain precautions. We have taken into consideration the fact that we are close to the fault. Can we do more? Yes, we can. There's always a balancing act between the economics, how much can you afford, and what you are willing to live with. It's a beautiful area, the weather is lovely, it's close to my work, that's where I live.
AC: Ann Kirmidjian teaches structural engineering at Stanford University. Professor, thank you.
Pr. AK: You're welcome.
AC: NPR's DAY TO DAY continues.

### Assistant: After the recent earthquake in China, structural engineer Ann Kiremidjian from Stanford University discusses the extent of structural damage. She identifies that many collapsed buildings had non-ductile concrete frames lacking sufficient steel reinforcement, both vertically and horizontally. She emphasizes the importance of earthquake-resistant design, mentioning how China has developed seismic codes and hazard maps over the years. All of China, including the Sichuan province where the earthquake occurred, is in a highly seismic area due to fault lines. Kiremidjian explains that inspecting buildings after earthquakes involves focusing on the rupture zone within the first 15-20 kilometers, which requires the involvement of numerous professional structural engineers. The interview also touches on building near earthquake faults, highlighting the trade-off between economic considerations and safety precautions.

### Human: Summarize the following text.

{text}

### Assistant:"""

PROMPT_TEMPLATE_2 = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Summarize the following dialogue provided the context.

1) Some context
{summary}

2) Dialogue:
{text}

### Assistant:"""

INTERMEDIATE_FOLDER = 'xgen_7b_2sl'

In [5]:
def result_from_output_sequence(output, n_shot = 2):
    
    # Un-prompt output
    delimiter = '### Assistant:'
    output = delimiter.join(output.split(delimiter)[1 + n_shot:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '\n<|endoftext|>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
    print('---- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, eos_token_id=50256, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

### >>> Falcon 7B Instruct

In [1]:
model_name = 'tiiuae/falcon-7b-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

NameError: name 'AutoTokenizer' is not defined

In [4]:
PROMPT_TEMPLATE_1 = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
>>QUESTION<<Summarize the following text.

{text}
>>ANSWER<<"""

INTERMEDIATE_FOLDER = 'falcon_7b_inst'

In [5]:
def result_from_output_sequence(output, n_shot = 0):
    
    # Un-prompt output
    delimiter = '>>ANSWER<<'
    output = delimiter.join(output.split(delimiter)[1 + n_shot:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '<|endoftext|>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_token_type_ids=False, return_tensors="pt").to('cuda')
    print('---- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

### >>> MPT-7B-Instruct

In [3]:
model_name = 'mosaicml/mpt-7b-instruct'

config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = 'cuda:0' # For fast initialization directly on GPU!

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

Instantiating an MPTForCausalLM model from /workspace/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/925e0d80e50e77aaddaf9c3ced41ca4ea23a1025/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NotImplementedError: The model type mpt is not yet supported to be used with BetterTransformer. Feel free to open an issue at https://github.com/huggingface/optimum/issues if you would like this model type to be supported. Currently supported models are: dict_keys(['albert', 'bark', 'bart', 'bert', 'bert-generation', 'blenderbot', 'bloom', 'camembert', 'blip-2', 'clip', 'codegen', 'data2vec-text', 'deit', 'distilbert', 'electra', 'ernie', 'fsmt', 'gpt2', 'gpt_bigcode', 'gptj', 'gpt_neo', 'gpt_neox', 'hubert', 'layoutlm', 'llama', 'm2m_100', 'marian', 'markuplm', 'mbart', 'opt', 'pegasus', 'rembert', 'prophetnet', 'roberta', 'roc_bert', 'roformer', 'splinter', 'tapas', 't5', 'vilt', 'vit', 'vit_mae', 'vit_msn', 'wav2vec2', 'whisper', 'xlm-roberta', 'yolos']).

In [ ]:
# Prompt template from HuggingFace. Dolly 15k
PROMPT_TEMPLATE_1 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Summarize the following text.
{text}
### Response:
"""

INTERMEDIATE_FOLDER = 'mpt_7b_inst'

In [ ]:
def result_from_output_sequence(output, n_shot = 0):
    
    # Un-prompt output
    delimiter = '###Response:\n'
    output = delimiter.join(output.split(delimiter)[1 + n_shot:])
    
    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]
    
    # Remove <|endoftext|>
    eos_token = '<|endoftext|>'
    output = output[:-len(eos_token)]
    
    return output

def generate_output(prompt, counter, chunks):
    input_ids = tokenizer(prompt, return_token_type_ids=False, return_tensors="pt").to('cuda')
    print('---- Computing chunk', counter, 'of', len(chunks), 'total, size:', len(input_ids['input_ids'][0]), 'tokens')
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
    return tokenizer.decode(output_ids[0]).strip()

# Pipeline

### Loading samples

In [8]:
def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

def get_intermediate_file_path(sample):
    global INTERMEDIATE_FOLDER

    return "intermediate/" + INTERMEDIATE_FOLDER + '/' + sample

# Load samples from dataset
samples = os.listdir('input/texts')
samples = [sample for sample in samples if sample.endswith('.txt')]
samples.sort()
n_samples = len(samples)
print('Found', n_samples, 'samples:', samples)

Found 30 samples: ['sample_10_9852.txt', 'sample_11_10498.txt', 'sample_12_11166.txt', 'sample_13_4562.txt', 'sample_14_6858.txt', 'sample_15_9989.txt', 'sample_16_6057.txt', 'sample_17_4135.txt', 'sample_18_9825.txt', 'sample_19_9648.txt', 'sample_1_4693.txt', 'sample_20_13365.txt', 'sample_21_7283.txt', 'sample_22_6522.txt', 'sample_23_9169.txt', 'sample_24_10638.txt', 'sample_25_2751.txt', 'sample_26_7860.txt', 'sample_27_9004.txt', 'sample_28_5705.txt', 'sample_29_3855.txt', 'sample_2_11567.txt', 'sample_30_8361.txt', 'sample_3_12019.txt', 'sample_4_13389.txt', 'sample_5_1343.txt', 'sample_6_10232.txt', 'sample_7_11826.txt', 'sample_8_7292.txt', 'sample_9_4578.txt']


### Chunking algorithm

In [9]:
MAX_CHARACTER_CHUNK_SIZE = 2000 * 3

def chunkize(text):
    """
    Greedy implementation of a dialogue transcript chunking algorithm. This method returns a list of transcript chunks.
    - It priorities stability over performance. There is a set maximum chunk size for LLM inference stability. Really long utterances bypass this limit.
    - It guarantees the cuts are made at utterance ends.

    !! WARNING !! The maximum size is in characters, not tokens. A good upper bound shall be used as there are
    no clear and easy correspondance between characters and tokens, as it obviously depends on the model tokenizer.

    Dialogue format recognized (used in the HF AMI dataset):
    ----------------------
    <A> Hello!
    <B> Hey there
    <A> What have you been up to
    <B> Not much honestly
    ----------------------
    """
    
    chunks = [] # Final list of transcript chunks. This makes up the loop invariant
    utterances = text.split('\n') # Transcript is split into sentences
    utterances.reverse() # Reverse everything!!
    current_chunk = ''

    # While there is still an utterance to process
    while len(utterances) > 0:
        utterance = utterances.pop()

        # Add to current chunk and proceed to next
        if len(current_chunk) + len(utterance) <= MAX_CHARACTER_CHUNK_SIZE:
            if len(current_chunk) > 0:
                current_chunk += '\n'
            current_chunk += utterance
        
        # Utterance is larger than maximum chunk size
        elif len(current_chunk) == 0:
            chunks.append(current_chunk)
            chunks.append(utterance)
            current_chunk = ''

        # Current chunk is big enough, append to list and create new one
        else:
            chunks.append(current_chunk)
            current_chunk = utterance

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

### Merging algorithm

In [10]:
def inference_sample(sample):
    
    # Current summary is the concatenation of all the chunk sub-summaries 
    current_summary = ''
    #previous_summary = ''
    chunks = chunkize(sample)

    # Split into chunks
    counter = 0
    for chunk in chunks:
        counter += 1

        # Create prompt for this chunk
        #if len(previous_summary) == 0:
        #    prompt = PROMPT_TEMPLATE_1.format(text=chunk)
        #    n_shot = 2
        #else:
        #    prompt = PROMPT_TEMPLATE_2.format(summary=previous_summary, text=chunk)
        #    n_shot = 0
        prompt = PROMPT_TEMPLATE_1.format(text=chunk)
        #n_shot = 0
            
        # Sample one sub-input
        output = generate_output(prompt, counter, chunks)
        subsummary = result_from_output_sequence(output)
        print(subsummary)

        # Add to summary
        if len(current_summary) > 0:
            current_summary += '\n\n'
        current_summary += subsummary
        #previous_summary = subsummary
    
    return current_summary

### Tests

In [9]:
# Tests for chunking algorithm
import os

sample_file_path = 'input/texts/' + samples[0]
sample_file = open(sample_file_path, 'r', encoding='utf-8')
sample = sample_file.read()
sample_file.close()

chunks = chunkize(sample)
for chunk in chunks:
    print('-----------------------------------------')
    print(chunk)

-----------------------------------------
<UI> Excellent.
<UI> So um I sent you the agenda, it was on the in the project documents.
<UI> I don't know if you got a chance to just have a look at it.
<UI> Anyway, it's the meeting's gonna follow more or less the same structure as last time, so we'll go round each of you in turn and you can give your presentations on what you've been up to.
<UI> Um and at the end of that we need to discuss what you've come up with, so that we can make a decision on the key remote control concepts, so that's we need to know about the components' properties, materials, the user interface and any trends that the Marketing Expert has been watching.
<ID> Right.
<UI> Okay.
<UI> Um, do you wanna start again?
<ID> Okay.
<UI> Let me we've got forty minutes.
<ID> Right s so I haven't made a PowerPoint presentation, yeah, I I thought I'll use the whiteboard instead.
<UI> You haven't made a PowerPoint, okay.
<UI> Mm-hmm.
<ID> Um mm, okay, so basically I'll start off by

# Inference

In [11]:
initial_time = time.time()
skipped_samples = 0

mkdir('intermediate')
mkdir('intermediate/' + INTERMEDIATE_FOLDER)

print('Starting computation...')

# For each sample in dataset
check = True
for sample_n in range(n_samples):
    if not(check):
        break

    # Estimate completion and time.
    cur_samples = sample_n - skipped_samples
    tot_samples = n_samples - skipped_samples
    progress = cur_samples / tot_samples
    pct = round(progress * 100, 1)
    print('Prompting on sample N' + str(sample_n + 1) + '/' + str(n_samples))
    print('-- Completion: ' + str(pct) + '%')
    if cur_samples > 0:
        approx_total = (time.time() - initial_time) / cur_samples * tot_samples
        approx_remaining = approx_total * (1 - progress)
        print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
    
    # Read sample and generate prompt
    sample_file_path = 'input/texts/' + samples[sample_n]
    sample_file = open(sample_file_path, 'r', encoding='utf-8')
    sample = sample_file.read()
    sample_file.close()
    
    # Find target file
    target_file_path = get_intermediate_file_path(samples[sample_n])
    if os.path.isfile(target_file_path):
        print('-- Found intermediate result file \'' + target_file_path + '\', skipped.')
        skipped_samples += 1
        continue

    try:
        check = False

        # Inference
        output = inference_sample(sample)

        # Save answer in file
        target_file = open(target_file_path, 'w', encoding='utf-8')
        target_file.write(output)
        target_file.close()
        
        check = True
    except:
        print('Error while generating sample')
        traceback.print_exc()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Starting computation...
Prompting on sample N1/30
-- Completion: 0.0%
---- Computing chunk 1 of 6 total, size: 1623 tokens


/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Error while generating sample
Done! Took 0:01:26 seconds


Traceback (most recent call last):
  File "/tmp/ipykernel_77989/3447844630.py", line 44, in <module>
    output = inference_sample(sample)
  File "/tmp/ipykernel_77989/1605117687.py", line 24, in inference_sample
    output = generate_output(prompt, counter, chunks)
  File "/tmp/ipykernel_77989/3332469274.py", line 20, in generate_output
    output_ids = model.generate(**input_ids, do_sample=DO_SAMPLE, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, top_k=TOP_K)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 1642, in generate
    return self.sample(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 2760, in sample
    next_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)
KeyboardInterrupt


# Calculs de scores

In [1]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------------------------- ---------- 1.1/1.5 MB 34.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 31.9 MB/s eta 0:00:00
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/1a/70/e63223f8116931d365993d4a6b7ef653a4d920b41d03de7c59499962821f/click-8.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10

In [10]:
import evaluate, os, statistics

In [18]:
# Methods and variables
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

#STORAGE_FILE_NAME = 'scores'

# Find output file for CSV scores
#mkdir('output')
#storage_file = open('output/' + STORAGE_FILE_NAME + '.csv', 'w', encoding='utf-8')
#storage_file.write('path;rouge2;rougel;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each sample in dataset
for sample_n in range(n_samples):

    # Find generated summary
    target_file_path = get_intermediate_file_path(samples[sample_n])
    if not os.path.isfile(target_file_path):
        print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
        continue

    # Read sample and generate prompt -> Keep summary
    summary_file_path = 'input/summaries/' + samples[sample_n]
    summary_file = open(summary_file_path, 'r', encoding='utf-8')
    references.append(summary_file.read())
    summary_file.close()

    # Access generated summary
    target_file = open(target_file_path, 'r', encoding='utf-8')
    prediction = target_file.read()
    target_file.close()

    # Add prediction
    target_file_paths.append(target_file_path)
    predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

# Calculate average
list_rouge1, list_rouge2, list_rougel, list_bertscore = result_rouge['rouge1'], result_rouge['rouge2'], result_rouge['rougeL'], result_bertscore['f1']

# Write to csv
# Forget about BLEU...
# Format: PATH | ROUGE1 | ROUGE2 | ROUGEL | BERTScore
#for i in range(len(target_file_paths)):
#    ligne = target_file_paths[i]
#
#    # list_rouge1[i]
#    ligne += ';' + str(list_rouge2[i]) + ";" + str(list_rougel[i])
#    #ligne += ";" + str(result_bleu['bleu'])
#    ligne += ";" + str(list_bertscore[i])
#    
#    storage_file.write(ligne + '\n')

#storage_file.close()
print('Done!')

# Calculate means
print('Average rouge1:', statistics.mean(list_rouge1))
print('Average rouge2:', statistics.mean(list_rouge2))
print('Average rougel:', statistics.mean(list_rougel))
print('Average bertscore:', statistics.mean(list_bertscore))

Starting scores computation...
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 29191.96 seconds, 0.00 sentences/sec
Done!
Average rouge1: 0.2661548987273394
Average rouge2: 0.07980427751892992
Average rougel: 0.15763943598433525
Average bertscore: 0.20706295669078828
